# Running on new images
This notebook will walk you step by step through the process of using a pre-trained model to detect traffic signs in an image.

# Imports

In [1]:
import warnings
# warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import os
import tensorflow as tf

tf.enable_eager_execution()


from matplotlib import pyplot as plt
from PIL import Image
import glob as glob
import random
import pickle

%matplotlib inline

In [2]:
tf.executing_eagerly()

True

# Environment setup

In [3]:

import sys

sys.path.append('../models/research')  # Replace with the path to TensorFlow Object Detection API
sys.path.append('../darkflow')  # Replace with the path to Darkflow

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util


# Tensorflow Object Detection API

## Model preparation 

In [4]:
# MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous'
# MODEL_NAME = 'faster_rcnn_resnet_101'
# MODEL_NAME = 'faster_rcnn_resnet50'
MODEL_NAME = 'faster_rcnn_inception_v2'
# MODEL_NAME = 'rfcn_resnet101'
# MODEL_NAME = 'ssd_inception_v2'
# MODEL_NAME = 'ssd_mobilenet_v1'

In [5]:
# Path to frozen detection graph. This is the actual model that is used for the traffic sign detection.
MODEL_PATH = os.path.join('models', MODEL_NAME)
PATH_TO_CKPT = os.path.join(MODEL_PATH,'inference_graph/frozen_inference_graph.pb')

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('gtsdb_data', 'gtsdb3_label_map.pbtxt')

NUM_CLASSES = 3


## Load a (frozen) Tensorflow model into memory

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `2`, we know that this corresponds to `mandatory`.

In [7]:
PATH_TO_LABELS = '/home/yuxing/my_project/traffic-sign-detection/gtsdb_data/gtsdb3_label_map.pbtxt'

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(label_map)



item {
  name: "prohibitory"
  id: 1
}
item {
  name: "mandatory"
  id: 2
}
item {
  name: "danger"
  id: 3
}



## Helper code

In [8]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

## Detection

In [9]:

PATH_TO_TEST_IMAGES_DIR = '/home/yuxing/my_project/traffic-sign-detection/test_images'

TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.jpg'))

# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [10]:
saved_detection_results = {}

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            image = Image.open(image_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            print(type(image_np_expanded))
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            
            # Save the detection results for this image
            saved_detection_results[image_path] = {
                'boxes': boxes,
                'scores': scores,
                'classes': classes,
                'num_detections': num_detections}

            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(image_np)

        

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [11]:

def perform_inference(sess, image_tensor, image_np_expanded, boxes, scores, classes, num_detections):
    print(f"Type of image_np_expanded: {type(image_np_expanded)}")
    print(f"Shape of image_np_expanded: {image_np_expanded.shape if isinstance(image_np_expanded, np.ndarray) else tf.shape(image_np_expanded)}")
    if tf.is_tensor(image_np_expanded):
        image_np_expanded = sess.run(image_np_expanded)
    return sess.run([boxes, scores, classes, num_detections], 
                    feed_dict={image_tensor: image_np_expanded})


def create_adversarial_pattern(input_image, input_label, sess):
    print(f"Type of input_image: {type(input_image)}")
    print(f"Shape of input_image: {input_image.shape if isinstance(input_image, np.ndarray) else tf.shape(input_image)}")
    print(f"Type of input_label: {type(input_label)}")
    if not tf.is_tensor(input_image):
        input_image = tf.convert_to_tensor(input_image.astype(np.float32))

    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = perform_inference(sess, image_tensor, input_image, boxes, scores, classes, num_detections)
        loss = loss_object(input_label, prediction[1]) 
    
    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    return signed_grad



loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for idx, image_path in enumerate(TEST_IMAGE_PATHS):
            # Get the saved results
            boxes = saved_detection_results[image_path]['boxes']
            scores = saved_detection_results[image_path]['scores']
            classes = saved_detection_results[image_path]['classes']
            
            # Find the detection with the highest score
            max_score_idx = np.argmax(scores)
            target_class = int(classes[0, max_score_idx])
            
            # Create one-hot label for the class
            label = tf.one_hot(target_class, NUM_CLASSES)
            label = tf.reshape(label, (1, NUM_CLASSES))
            
            # Prepare the image
            image = Image.open(image_path)
            image_np = np.array(image).astype(np.uint8)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            input_image = tf.convert_to_tensor(image_np_expanded.astype(np.float32))
            
            
            # Generate the adversarial example
            print(f"Type of input_image before calling create_adversarial_pattern: {type(input_image)}")
            print(f"Shape of input_image before calling create_adversarial_pattern: {input_image.shape if isinstance(input_image, np.ndarray) else tf.shape(input_image)}")         
            perturbations = create_adversarial_pattern(input_image, label, sess)
            
            # Create the adversarial image
            eps = 0.1
            adv_x = image_np + eps * perturbations.numpy()
            adv_x = np.clip(adv_x, 0, 255).astype(np.uint8)
            
            # Run inference on the adversarial image
            (adv_boxes, adv_scores, adv_classes, adv_num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: np.expand_dims(adv_x, axis=0)})
            
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                adv_x,
                np.squeeze(adv_boxes),
                np.squeeze(adv_classes).astype(np.int32),
                np.squeeze(adv_scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)
            
            plt.figure(idx, figsize=IMAGE_SIZE)
            plt.axis('off')
            plt.imshow(adv_x)
            plt.show()


Type of input_image before calling create_adversarial_pattern: <class 'tensorflow.python.framework.ops.Tensor'>
Shape of input_image before calling create_adversarial_pattern: Tensor("Shape_8:0", shape=(4,), dtype=int32)
Type of input_image: <class 'tensorflow.python.framework.ops.Tensor'>
Shape of input_image: Tensor("Shape_9:0", shape=(4,), dtype=int32)
Type of input_label: <class 'tensorflow.python.framework.ops.Tensor'>
Type of image_np_expanded: <class 'tensorflow.python.framework.ops.Tensor'>
Shape of image_np_expanded: Tensor("Shape_10:0", shape=(4,), dtype=int32)


TypeError: Fetch argument array([[[0.4290882 , 0.548742  , 0.49913642, 0.6064588 ],
        [0.43641028, 0.55133986, 0.50232947, 0.6063809 ],
        [0.43638423, 0.55540466, 0.49756598, 0.5936275 ],
        ...,
        [0.9616194 , 0.2903184 , 0.9967665 , 0.3395127 ],
        [0.9608694 , 0.23064701, 0.99679893, 0.27922806],
        [0.96051943, 0.21056695, 0.99688154, 0.2592434 ]]], dtype=float32) has invalid type <class 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)

In [ ]:
for op in detection_graph.get_operations():
    if 'label' in op.name.lower():
        print(op.name)
